In [19]:
!pip install transformers

In [3]:
from tqdm.notebook import tqdm
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as TorchDataset
import torch.nn as nn

In [4]:
!pip install jsonlines
import jsonlines
from sklearn.model_selection import train_test_split
!pip install python-levenshtein
import Levenshtein as Lev
import torch
from torch.utils.data import Dataset as TorchDataset
from typing import Sequence, Dict, Any, List
import json

     |████████████████████████████████| 51kB 5.9MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144802 sha256=d6e22374c9ceb76ce16a55bb6f968ab765e9c952ee6d124f0b8ea300d2a15bf0
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein


In [20]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [23]:
train_path = "/content/gdrive/My Drive/train_dl.json"
test_path = "/content/gdrive/My Drive/test_dl.json"

In [25]:
def load_jsonlines(path: str) -> List[Dict[str, Any]]:
    data = []
    with jsonlines.open(path, "r") as reader:
        for items in reader:
            data.append(items)
    return data

In [26]:
train = load_jsonlines(train_path)
test = load_jsonlines(test_path)

In [27]:
len(train)

26422

In [31]:
for i in test:
    print(i)
    break

{'seq_a': 'what are amphibians?', 'seq_b': 'the president was unmarried', 'dist': 4}


In [32]:
class DLDataset(TorchDataset):

    def __init__(self, data, index=0, batch_size=1):
        self.data = data
        self.index = index
        self.batch_size = batch_size

    def __getitem__(self, data):
        global index
        data = self.data
        batch_size = self.batch_size
        a_list, b_list, dist_list = [], [], []

        for _ in range(batch_size):
            word_a = self.data[self.index]['seq_a']
            word_a = torch.Tensor(tokenizer.encode(word_a, padding='max_length', truncation=True)).unsqueeze(0)
            word_a[word_a == 101] = 0
            
            word_b = self.data[self.index]['seq_b']
            word_b = torch.Tensor(tokenizer.encode(word_b, padding='max_length', truncation=True)).unsqueeze(0)
            word_b[word_b == 102] = 0
            word_dis = self.data[self.index]['dist']


            word_dis= torch.Tensor([word_dis]).unsqueeze(0)
            a_list.append(word_a)
            b_list.append(word_b)
            dist_list.append(word_dis)
            self.index = (self.index + 1) % len(data)

        tens_a = torch.cat(a_list, dim=0)
        tens_b = torch.cat(b_list, dim=0)
        tens_dis = torch.cat(dist_list, dim=0)
        return tens_a, tens_b, tens_dis

    def __len__(self):
        return len(self.data) // self.batch_size

In [34]:
index = 0 
initial_ = DLDataset(train)
loader = DataLoader(initial_, batch_size=64, shuffle=True)

In [35]:
initial_val = DLDataset(test)
val_loader = DataLoader(initial_val, batch_size=64, shuffle=True)

In [36]:
class Deep_lev(torch.nn.Module):

    def __init__(self, vocab_size=30522, embedding_dim=128, hidden_dim=128) :
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.encoder = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        self.linear = nn.Linear(hidden_dim * 3, 1)


    def encode_sequence(self, sequence):
        embedded_sequence = self.embeddings(sequence)
        out, (ht, ct) = self.encoder(embedded_sequence)
        return ht[-1]

    def forward(self, sequence_a, sequence_b, distance = None):
        embedded_sequence_a = self.encode_sequence(sequence_a)
        embedded_sequence_b = self.encode_sequence(sequence_b)
        diff = torch.abs(embedded_sequence_a - embedded_sequence_b)
        representation = torch.cat([embedded_sequence_a, embedded_sequence_b, diff], dim=-1)

        approx_distance = self.linear(representation)

        return approx_distance

In [39]:
deep_lv = Deep_lev().to(device)
n_epochs = 40
opt = torch.optim.Adam(deep_lv.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, n_epochs)
loss = nn.MSELoss()
losses = []
val_losses = []

for epoch in range(n_epochs):
    deep_lv.train()
    epoch_loss = 0
    for batch in loader:
        opt.zero_grad()
        preds = deep_lv(batch[0].squeeze(1).long().to(device), batch[1].squeeze(1).long().to(device))
        loss_ = loss(preds.squeeze(1), batch[2].squeeze(2).squeeze(1).to(device))
        loss_.backward()
        epoch_loss += loss_.item()
        opt.step()

    losses.append(epoch_loss / len(loader))

    deep_lv.eval()
    with torch.no_grad():
        average_loss = 0
        for batch in val_loader:
            preds = deep_lv(batch[0].squeeze(1).long().to(device), batch[1].squeeze(1).long().to(device))
            loss_ = loss(preds.squeeze(1), batch[2].squeeze(2).squeeze(1).to(device))
            average_loss += loss_.item()
    val_losses.append(average_loss / len(val_loader))

    scheduler.step()
    print(f"Epoch: {epoch} train_loss: {(epoch_loss / len(loader))} val_loss: {(average_loss / len(val_loader))}")

    if epoch >= 20:
        if (epoch_loss / len(loader)) >= losses[-1] and (epoch_loss / len(loader)) >= losses[-2]:
            print('early stopping')
            torch.save(deep_lv.state_dict(), "/content/gdrive/My Drive/Deep_levenstein.pt")
            break

Epoch: 0 train_loss: 9.129678009208698 val_loss: 8.814227877878675
Epoch: 1 train_loss: 8.858582776626143 val_loss: 8.741375593578114
Epoch: 2 train_loss: 8.849247110380675 val_loss: 8.724271757929932
Epoch: 3 train_loss: 8.845117969605305 val_loss: 8.717440095602297
Epoch: 4 train_loss: 8.8414488635398 val_loss: 8.699610525486516
Epoch: 5 train_loss: 8.835635891260882 val_loss: 8.699093806977366
Epoch: 6 train_loss: 8.835313112337422 val_loss: 8.697882818240746
Epoch: 7 train_loss: 8.834526549817285 val_loss: 8.69588074497148
Epoch: 8 train_loss: 8.833547798253722 val_loss: 8.692605063027026
Epoch: 9 train_loss: 8.831733461153709 val_loss: 8.683570394328996
Epoch: 10 train_loss: 8.830540236780198 val_loss: 8.677434283144335
Epoch: 11 train_loss: 8.829395302271438 val_loss: 8.670688753034554
Epoch: 12 train_loss: 8.82810860800108 val_loss: 8.664206264065761
Epoch: 13 train_loss: 8.826503899137853 val_loss: 8.658889931790968
Epoch: 14 train_loss: 8.824271107412713 val_loss: 8.6553440023

In [40]:
torch.save(deep_lv.state_dict(), "/content/gdrive/My Drive/Deep_levenstein_40epoch.pt")

In [ ]:
deep_lv = Deep_lev().to(device)
deep_lv.train()
opt = torch.optim.Adam(deep_lv.parameters(), lr=0.001)
loss = nn.MSELoss()
losses = []
n_epochs = 50
for epoch in range(n_epochs):
    epoch_loss = 0
    for batch in loader:
        opt.zero_grad()
        preds = deep_lv(batch[0].squeeze(1).long().to(device), batch[1].squeeze(1).long().to(device))
        loss_ = loss(preds.squeeze(1), batch[2].squeeze(2).squeeze(1).to(device))
        loss_.backward()
        epoch_loss += loss_.item()
        opt.step()

    losses.append(epoch_loss / len(loader))
    print(f"Epoch: {epoch} train_loss: {(epoch_loss / len(loader))}")
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model
    if epoch >= 20:
        if (epoch_loss / len(loader)) >= losses[-1] and (epoch_loss / len(loader)) >= losses[-2]:
            print('early stopping')
            torch.save(deep_lv.state_dict(), 'Deep_lev.pt')
            break

Epoch: 0 train_loss: 7.709730357626756
Epoch: 1 train_loss: 7.221348355639095
Epoch: 2 train_loss: 7.2186108217071805
Epoch: 3 train_loss: 7.216133877901604
Epoch: 4 train_loss: 7.215924262076902
Epoch: 5 train_loss: 7.215797187459355
Epoch: 6 train_loss: 7.215193408841904
Epoch: 7 train_loss: 7.214864653510788
Epoch: 8 train_loss: 7.214100389815748
Epoch: 9 train_loss: 7.213612308050878


In [41]:
torch.save(deep_lv.state_dict(), '/content/gdrive/My Drive/Deep_levenstein_40epoch.pt')

In [42]:
model_prov = Deep_lev().to(device)

In [43]:
model_prov.load_state_dict(torch.load('/content/gdrive/My Drive/Deep_levenstein_40epoch.pt'))

<All keys matched successfully>

In [44]:
index = 0 
initial_test = DLDataset(test)
test_loader = DataLoader(initial_test, batch_size=64, shuffle=True)

In [46]:
loss = nn.MSELoss()
average_loss = 0
model_prov.eval()
with torch.no_grad():
    for batch in test_loader:
        preds = model_prov(batch[0].squeeze(1).long().to(device), batch[1].squeeze(1).long().to(device))
        loss_ = loss(preds.squeeze(1), batch[2].squeeze(2).squeeze(1).to(device))
        average_loss += loss_.item()
        
    print("L2 distance on test:", average_loss / len(test_loader))

L2 distance on test: 8.645169302528979
